In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem
import nest_asyncio
nest_asyncio.apply()
# setting up api key
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
import wikipedia
import openmeteo_requests
import requests_cache
# from retry_requests import retry
import requests
from agents.extensions.visualization import draw_graph

import json 
import requests
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")


from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

from tools import flight_search, yelp_search, hotels_search, wikipedia_search, get_weather
from pydantic import BaseModel
from models import first_agent_output
from agents.model_settings import ModelSettings
from data_models.flight_data_models import FlightSearchResults, BestFlight, AirportInfo, Flight, Layover, OtherFlight, PriceInsights, AirportLocation, Airports

from travel_agents import question_rewrite_agent, flight_agent
from functions import flight_search
from pprint import pprint

END OF THE SCRIPT


In [2]:
ank = "ankara(ESB)"
ist = "istanbul(SAW)"
amst = "amsterdam(AMS)"
pekin = "Peking(PEK)"

start_point = ank 
dest_point = pekin

main_question = f"""I want to travel from {start_point} to {dest_point}. The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
I also need a hotel to stay. I do not like smell of cigarette. If possible, I want to see the rooms where smoking is not allowed. 
In addition to that, I like eating sea food. I guess {dest_point} is very rich city interms of sea restaurants""".strip()

question_rewrite_agent_output = Runner.run_sync(question_rewrite_agent,main_question)

In [3]:
question_rewrite_agent_output.final_output

FinalModel(flight_reqs=FlightOutput(departure_id='ESB', arrival_id='PEK', outbound_data='2025-05-05', return_data='2025-05-11'), yelp_reqs=YelpOutput(search_term='sea food', location='Peking'), hotel_reqs=HotelOutput(query='Peking', check_in_date='2025-05-05', check_out_date='2025-05-11', gl='non-smoking'))

In [4]:
question_rewrite_agent_output.final_output

FinalModel(flight_reqs=FlightOutput(departure_id='ESB', arrival_id='PEK', outbound_data='2025-05-05', return_data='2025-05-11'), yelp_reqs=YelpOutput(search_term='sea food', location='Peking'), hotel_reqs=HotelOutput(query='Peking', check_in_date='2025-05-05', check_out_date='2025-05-11', gl='non-smoking'))

In [5]:
question_rewrite_agent_output.final_output.flight_reqs.return_data

'2025-05-11'

In [6]:
# flight_results = Runner.run_sync(flight_agent,main_question)

In [7]:
x = flight_search(
    # departure_id= "ESB",
    # arrival_id='AMS',
    departure_id=question_rewrite_agent_output.final_output.flight_reqs.departure_id, 
    arrival_id= question_rewrite_agent_output.final_output.flight_reqs.arrival_id ,
    outbound_data= question_rewrite_agent_output.final_output.flight_reqs.outbound_data, 
    return_date = question_rewrite_agent_output.final_output.flight_reqs.return_data
    )

flight_results = FlightSearchResults.parse_obj(x)  # Use parse_obj with dict


C:\Users\user\AppData\Local\Temp\ipykernel_17968\3572277655.py:10: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  flight_results = FlightSearchResults.parse_obj(x)  # Use parse_obj with dict


In [8]:
x

{'search_metadata': {'id': '67fd7e8f13ac98c1f43c7e35',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/12487f80c39b39af/67fd7e8f13ac98c1f43c7e35.json',
  'created_at': '2025-04-14 21:30:55 UTC',
  'processed_at': '2025-04-14 21:30:55 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI1LTA1LTA1agcIARIDRVNCcgcIARIDUEVLGh4SCjIwMjUtMDUtMTFqBwgBEgNQRUtyBwgBEgNFU0JCAQFIAXABmAEB&tfu=EgIIAQ',
  'raw_html_file': 'https://serpapi.com/searches/12487f80c39b39af/67fd7e8f13ac98c1f43c7e35.html',
  'prettify_html_file': 'https://serpapi.com/searches/12487f80c39b39af/67fd7e8f13ac98c1f43c7e35.prettify',
  'total_time_taken': 1.04},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'departure_id': 'ESB',
  'arrival_id': 'PEK',
  'outbound_date': '2025-05-05',
  'return_date': '2025-05-11',
  'currency': 'USD',
  'deep_search': True},
 'best_flights': [{'flights': [{'departure_airport': {'name'

In [9]:
x.keys()

dict_keys(['search_metadata', 'search_parameters', 'best_flights', 'other_flights', 'price_insights', 'airports'])

In [10]:
x['other_flights']

[{'flights': [{'departure_airport': {'name': 'Ankara Esenboga Airport',
     'id': 'ESB',
     'time': '2025-05-05 11:00'},
    'arrival_airport': {'name': 'Istanbul Airport',
     'id': 'IST',
     'time': '2025-05-05 12:25'},
    'duration': 85,
    'airplane': 'Boeing 737',
    'airline': 'Turkish Airlines',
    'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/TK.png',
    'travel_class': 'Economy',
    'flight_number': 'TK 2135',
    'ticket_also_sold_by': ['LOT'],
    'legroom': '31 in',
    'extensions': ['Average legroom (31 in)',
     'Wi-Fi for a fee',
     'In-seat USB outlet',
     'On-demand video',
     'Carbon emissions estimate: 67 kg']},
   {'departure_airport': {'name': 'Istanbul Airport',
     'id': 'IST',
     'time': '2025-05-06 07:00'},
    'arrival_airport': {'name': 'Warsaw Frederic Chopin',
     'id': 'WAW',
     'time': '2025-05-06 08:30'},
    'duration': 150,
    'airplane': 'Embraer 195',
    'airline': 'LOT',
    'airline_logo': 'https://

In [43]:
def flight2str(x, flight_key):
    # flight = OtherFlight(**x['other_flights'][1])
    # flight = OtherFlight(**x['other_flights'][1])

    text_summary = ""
    for outer_index in range(len(x[flight_key])):  # Renamed the outer loop variable
        flight = OtherFlight(**x[flight_key][outer_index])
        # Loop through flights and embed layovers

        text_summary += f"""\n*****Fligth: {outer_index + 1}*****\n"""
        text_summary += f"""
Flight Summary:
Type: {flight.type}
Total Duration: {flight.total_duration} minutes
Price: ${flight.price}"""
        for inner_index, f in enumerate(flight.flights):  # Renamed the inner loop variable
            text_summary += f"""
        --- Segment {inner_index + 1} ---
        Airline: {f.airline or 'N/A'} ({f.flight_number})
        From: {f.departure_airport.name} ({f.departure_airport.id}) at {f.departure_airport.time}
        To: {f.arrival_airport.name} ({f.arrival_airport.id}) at {f.arrival_airport.time}
        Duration: {f.duration} minutes
        Class: {f.travel_class}
        """

            # Only insert a layover if one exists after this flight
            if flight.layovers and inner_index < len(flight.layovers):
                layover = flight.layovers[inner_index]
                text_summary += f"Layover at {layover.name} ({layover.id}) - {layover.duration} minutes\n"

    return text_summary

In [48]:
flight_info = flight2str(x,flight_key='best_flights')

In [49]:
print(flight_info)


*****Fligth: 1*****

Flight Summary:
Type: Round trip
Total Duration: 1365 minutes
Price: $1387
        --- Segment 1 ---
        Airline: SunExpress (XQ 714)
        From: Ankara Esenboga Airport (ESB) at 2025-05-05 08:00
        To: Munich International Airport (MUC) at 2025-05-05 10:10
        Duration: 190 minutes
        Class: Economy
        Layover at Munich International Airport (MUC) - 570 minutes

        --- Segment 2 ---
        Airline: Lufthansa (LH 722)
        From: Munich International Airport (MUC) at 2025-05-05 19:40
        To: Beijing Capital International Airport (PEK) at 2025-05-06 11:45
        Duration: 605 minutes
        Class: Economy
        
*****Fligth: 2*****

Flight Summary:
Type: Round trip
Total Duration: 1365 minutes
Price: $1636
        --- Segment 1 ---
        Airline: Turkish Airlines (TK 2139)
        From: Ankara Esenboga Airport (ESB) at 2025-05-05 12:00
        To: Istanbul Airport (IST) at 2025-05-05 13:20
        Duration: 80 minutes
    

In [46]:
f_agent = Agent(name="Flight Assistant agent",
              instructions=f"You are helpful flight asisstants try to answer the questions according to: \n\n{flight_info}. You are not allowed to recommend or answer any other thing than flights",
              model="gpt-4o-mini",
              # model = 'gpt-4o',
              model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*8),
              )

In [47]:
result = Runner.run_sync(f_agent, main_question+" bring me cheapest flight, also tell me how much does that cost?")
print(result.final_output)

Here are the flight options for your trip from Ankara (ESB) to Beijing (PEK) on the outbound date of 2025-05-05 and returning on 2025-05-11:

### Cheapest Flight Option:
**Flight: 4**
- **Total Duration:** 1540 minutes
- **Price:** $None (This indicates that the flight is free, but please verify the details)
  
#### Flight Segments:
1. **Turkish Airlines (TK 2117)**
   - From: Ankara Esenboga Airport (ESB) at 2025-05-05 08:00
   - To: Istanbul Airport (IST) at 2025-05-05 09:25
   - Duration: 85 minutes
   - Class: Economy
   - Layover at Istanbul Airport (IST) - 235 minutes

2. **Singapore Airlines (SQ 391)**
   - From: Istanbul Airport (IST) at 2025-05-05 13:20
   - To: Singapore Changi Airport (SIN) at 2025-05-06 04:55
   - Duration: 635 minutes
   - Class: Premium Economy
   - Layover at Singapore Changi Airport (SIN) - 215 minutes

3. **Singapore Airlines (SQ 802)**
   - From: Singapore Changi Airport (SIN) at 2025-05-06 08:30
   - To: Beijing Capital International Airport (PEK) at

In [22]:
result = Runner.run_sync(f_agent, main_question+" what is the shortest flight, how long does it take?")
print(result.final_output)

For your travel from Ankara (ESB) to Beijing (PEK) on 2025-05-05, the shortest flight option is:

### Flight Option:
**Flight: 4**
- **Segment 1:**
  - **Airline:** Turkish Airlines (TK 2117)
  - **From:** Ankara Esenboga Airport (ESB) at 2025-05-05 08:00
  - **To:** Istanbul Airport (IST) at 2025-05-05 09:25
  - **Duration:** 85 minutes
- **Segment 2:**
  - **Airline:** Singapore Airlines (SQ 391)
  - **From:** Istanbul Airport (IST) at 2025-05-05 13:20
  - **To:** Singapore Changi Airport (SIN) at 2025-05-06 04:55
  - **Duration:** 635 minutes
- **Segment 3:**
  - **Airline:** Singapore Airlines (SQ 802)
  - **From:** Singapore Changi Airport (SIN) at 2025-05-06 08:30
  - **To:** Beijing Capital International Airport (PEK) at 2025-05-06 14:40
  - **Duration:** 370 minutes

### Total Duration:
- **Total Flight Duration:** 85 + 635 + 370 = **1090 minutes** (approximately 18 hours and 10 minutes).

This is the shortest flight option available based on the provided flights. 

For hotel a

In [23]:
x.keys()

dict_keys(['search_metadata', 'search_parameters', 'best_flights', 'other_flights', 'price_insights', 'airports'])

In [26]:
len(x['best_flights'])

3

In [29]:
len(x['price_insights']['price_history'])

21

In [30]:
len(x['other_flights'])

4

In [32]:
x

{'search_metadata': {'id': '67fd7e8f13ac98c1f43c7e35',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/12487f80c39b39af/67fd7e8f13ac98c1f43c7e35.json',
  'created_at': '2025-04-14 21:30:55 UTC',
  'processed_at': '2025-04-14 21:30:55 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI1LTA1LTA1agcIARIDRVNCcgcIARIDUEVLGh4SCjIwMjUtMDUtMTFqBwgBEgNQRUtyBwgBEgNFU0JCAQFIAXABmAEB&tfu=EgIIAQ',
  'raw_html_file': 'https://serpapi.com/searches/12487f80c39b39af/67fd7e8f13ac98c1f43c7e35.html',
  'prettify_html_file': 'https://serpapi.com/searches/12487f80c39b39af/67fd7e8f13ac98c1f43c7e35.prettify',
  'total_time_taken': 1.04},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'departure_id': 'ESB',
  'arrival_id': 'PEK',
  'outbound_date': '2025-05-05',
  'return_date': '2025-05-11',
  'currency': 'USD',
  'deep_search': True},
 'best_flights': [{'flights': [{'departure_airport': {'name'

In [39]:
for i in range(len(x['best_flights'])):

    flight = BestFlight(**x['best_flights'][i])

    text_summary = f"""
    ✈️ Flight Summary:
    Type: {flight.type}
    Total Duration: {flight.total_duration} minutes
    Price: ${flight.price}
    Airline Logo: {flight.airline_logo or 'N/A'}
    Carbon Emissions: {flight.carbon_emissions.this_flight / 1000:.1f} kg (Typical: {flight.carbon_emissions.typical_for_this_route / 1000:.1f} kg, Difference: {flight.carbon_emissions.difference_percent}%)
    """

    # Loop through flights and embed layovers
    for i, f in enumerate(flight.flights):
        text_summary += f"""
    --- Segment {i + 1} ---
    Airline: {f.airline or 'N/A'} ({f.flight_number})
    From: {f.departure_airport.name} ({f.departure_airport.id}) at {f.departure_airport.time}
    To: {f.arrival_airport.name} ({f.arrival_airport.id}) at {f.arrival_airport.time}
    Duration: {f.duration} minutes
    Airplane: {f.airplane}
    Class: {f.travel_class}
    Legroom: {f.legroom or 'N/A'}
    Extensions: {', '.join(f.extensions or [])}
    """

        # Only insert a layover if one exists after this flight
        if flight.layovers and i < len(flight.layovers):
            layover = flight.layovers[i]
            text_summary += f"Layover at {layover.name} ({layover.id}) - {layover.duration} minutes\n"

    text_summary += f"\nDeparture Token: {flight.departure_token}"

    print(text_summary)


    ✈️ Flight Summary:
    Type: Round trip
    Total Duration: 1365 minutes
    Price: $1387
    Airline Logo: https://www.gstatic.com/flights/airline_logos/70px/multi.png
    Carbon Emissions: 592.0 kg (Typical: 615.0 kg, Difference: -4%)
    
    --- Segment 1 ---
    Airline: SunExpress (XQ 714)
    From: Ankara Esenboga Airport (ESB) at 2025-05-05 08:00
    To: Munich International Airport (MUC) at 2025-05-05 10:10
    Duration: 190 minutes
    Airplane: Boeing 737
    Class: Economy
    Legroom: 29 in
    Extensions: Below average legroom (29 in), Carbon emissions estimate: 154 kg
    Layover at Munich International Airport (MUC) - 570 minutes

    --- Segment 2 ---
    Airline: Lufthansa (LH 722)
    From: Munich International Airport (MUC) at 2025-05-05 19:40
    To: Beijing Capital International Airport (PEK) at 2025-05-06 11:45
    Duration: 605 minutes
    Airplane: Airbus A350
    Class: Economy
    Legroom: 31 in
    Extensions: Average legroom (31 in), Wi-Fi for a fee, I

In [34]:
x.keys()

dict_keys(['search_metadata', 'search_parameters', 'best_flights', 'other_flights', 'price_insights', 'airports'])

In [50]:
len(x['other_flights'])

4